# Social media sentiment trend's influence on Stock Market values

# Data description:
Project conclusion will be based on comparison of two different graphs with different datasets.
First: Popular stock market indeces like S&P500, APPL ( what you see on graph), <br>
Second: Users tweets sorted by special stock market related hashtag(like APPL)<br>
<B>SECOND DATASET REQUIRES WEB SCRAPPING AND IS IN PROGRESS(i didn't think it would be so hard)<B>

<h2>Data Variables</h2><br>
Only for first dataset for now:
<ul>
<li>Date - Day of trading</li>
<li>Open - stock price at the open</li>
<li>High - highest value of stock price on that day</li>
<li>Low - lowest value of stock price on that day</li>
<li>Close* - Closing price of the stock on that day</li>
<li>Adj Close - adjusted close price adjusted for both dividends and splits</li>
<li>Volume - number of shares that traded on that day</li>
</ul>

### Research Questions
<ol>
<li>Analyze correlation between Social media sentiment and Stock market(if there any)</li><li>Find the most active day both for trading and social media posting.</li>
<li>Determine the most popular stocks on the Social Media.</li>
<li>Figure out how social media affects stock market.</li>    
<li>Find what type of sentiment(negative/positive/neutral) is prevalent in stock market related social media</li>
</ol>    

## Data manipulation: cleaning and shaping

<b>Merging 2 dataset with applying stock name to them:</b><br>
Showing first 255 rows

In [1]:
import pandas as pd

#import plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import psutil

#import matplotlib
import matplotlib.pyplot as plt

In [2]:


df1 = pd.read_csv(r"AAPL.csv")
df2 = pd.read_csv(r"SP500INDEX.csv")
df3 = pd.read_csv(r"TSLA.csv")
merged = pd.concat([df1,df2],ignore_index=True)
del df1['Adj Close']
del df2['Adj Close']
del df3['Adj Close']

merged['Stock'] = 'null'
merged.loc[:252,'Stock'] = 'APPL'
merged.loc[253:506,'Stock'] = 'S&P500'
merged.drop(columns = ['Adj Close'],axis=1,inplace = True)
merged.iloc[:255]

,Date,Open,High,Low,Close,Volume,Stock
0,2019-11-05,64.262497,64.547501,64.080002,64.282501,79897600,APPL
1,2019-11-06,64.192497,64.372498,63.842499,64.309998,75864400,APPL
2,2019-11-07,64.684998,65.087502,64.527496,64.857498,94940400,APPL
3,2019-11-08,64.672501,65.110001,64.212502,65.035004,69986400,APPL
4,2019-11-11,64.574997,65.617500,64.570000,65.550003,81821200,APPL
...,...,...,...,...,...,...,...
250,2020-11-02,109.110001,110.680000,107.320000,108.769997,122866900,APPL
251,2020-11-03,109.660004,111.489998,108.730003,110.440002,107624400,APPL
252,2020-11-04,114.139999,115.589996,112.349998,114.949997,137809900,APPL
253,2019-11-05,3080.800049,3083.949951,3072.149902,3074.620117,4486130000,S&P500


In [3]:
merged['Volume'] = merged['Volume'].div(1_000_000) #Python trick with number formatting 100_000_000 = 1000000000
merged.rename(columns = {'Volume' : 'Volume(mil.)'},inplace= True)
merged

,Date,Open,High,Low,Close,Volume(mil.),Stock
0,2019-11-05,64.262497,64.547501,64.080002,64.282501,79.8976,APPL
1,2019-11-06,64.192497,64.372498,63.842499,64.309998,75.8644,APPL
2,2019-11-07,64.684998,65.087502,64.527496,64.857498,94.9404,APPL
3,2019-11-08,64.672501,65.110001,64.212502,65.035004,69.9864,APPL
4,2019-11-11,64.574997,65.617500,64.570000,65.550003,81.8212,APPL
...,...,...,...,...,...,...,...
501,2020-10-29,3277.169922,3341.050049,3259.820068,3310.110107,4903.0700,S&P500
502,2020-10-30,3293.590088,3304.929932,3233.939941,3269.959961,4840.4500,S&P500
503,2020-11-02,3296.199951,3330.139893,3279.739990,3310.239990,4310.5900,S&P500
504,2020-11-03,3336.250000,3389.489990,3336.250000,3369.159912,4220.0700,S&P500


<b>Stock Volume mean:</b>

In [4]:
merged.groupby('Stock').agg({'Volume(mil.)':'mean'})

,Volume(mil.)
Stock,
APPL,155.441230
S&P500,4731.604545


### Some future visualization

In [34]:
df1['Change'] = (((df1['Open']-df1['Close'])/ df1['Open']) * 100)
df2['Change'] = (((df1['Open']-df1['Close'])/ df1['Open']) * 100)
# df1.query('Change > 1',inplace = True)
fig = make_subplots(specs = [[{"secondary_y":True}]])
fig.add_trace(go.Scatter(x = df1['Date'],y = df1["Open"],name = 'APPL_sentiment'),secondary_y=True)
fig.add_trace(go.Scatter(x = df2['Date'],y = df2["Open"],name = 'S&P500'))
fig.update_layout(paper_bgcolor = 'rgba(0,0,0,0)',plot_bgcolor = 'rgba(0,0,0,0)')
fig.show()

### Reading scrapped data in csv format:

In [6]:
aapl_tweets = pd.read_csv(r"aapl_tweets.csv",header = 0,engine = 'python',encoding = 'UTF-8')
tsla_tweets = pd.read_csv(r"tsla_tweets.csv",header = 0,engine = 'python',encoding = 'UTF-8')
fb_tweets = pd.read_csv(r"fb_tweets.csv",header = 0,engine = 'python',encoding = 'UTF-8')
googl_tweets = pd.read_csv(r"googl_tweets.csv",header = 0,engine = 'python',encoding = 'UTF-8')
spx_tweets = pd.read_csv(r"spx_tweets.csv",header = 0,engine = 'python',encoding = 'UTF-8')

#### Deleting NULL columns (Scrapped automaticaly) <br> Displaying average row numbers in all datasets:

In [38]:
aapl_tweets = aapl_tweets.dropna(how='all',axis='columns')
tsla_tweets = tsla_tweets.dropna(how='all',axis='columns')
fb_tweets = fb_tweets.dropna(how='all',axis='columns')
googl_tweets = googl_tweets.dropna(how='all',axis='columns')
spx_tweets = spx_tweets.dropna(how='all',axis='columns')
mean_length = (len(aapl_tweets) + len(tsla_tweets) + len(fb_tweets) + len(googl_tweets) + len(spx_tweets)) / 5
mean_length

13423.2

#### VADER (Valence Aware Dictionary and sEntiment Reasoner) is a lexicon <br>and rule-based sentiment analysis tool that is specifically attuned to sentiments expressed in social media.

In [8]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyser = SentimentIntensityAnalyzer()

#### Adding columns describing polarity(sentiment) of tweet:

In [9]:
aapl_tweets['rating'] = [analyser.polarity_scores(x)['compound'] for x in aapl_tweets['tweet']]
aapl_tweets['neu'] = [analyser.polarity_scores(x)['neu'] for x in aapl_tweets['tweet']]
aapl_tweets['pos'] = [analyser.polarity_scores(x)['pos'] for x in aapl_tweets['tweet']]
aapl_tweets['neg'] = [analyser.polarity_scores(x)['neg'] for x in aapl_tweets['tweet']]

aapl_tweets[['id','date','tweet','user_id','username','rating']]

,id,date,tweet,user_id,username,rating
0,1324082507441147906,2020-11-04 20:14:04,I’ll do a video after the close about how do u...,455824771,RedDogT3,0.6597
1,1324082082985947142,2020-11-04 20:12:23,Tech stocks among those surging post election....,25360971,PowerLunch,0.3818
2,1324071060220727296,2020-11-04 19:28:35,Have you guys met our options trader ANDY? He...,1271899909675859968,MrFourToEight,0.6739
3,1324069281735176193,2020-11-04 19:21:31,Remember when people freaked a bit when $AAPL ...,100024370,OphirGottlieb,-0.3736
4,1324066749642399744,2020-11-04 19:11:27,$AAPL https://t.co/E70iPUMuqJ,1084979915072700422,BBGMarket,0.0000
...,...,...,...,...,...,...
12448,1288991426848731137,2020-07-31 00:14:39,$FB $AAPL $GOOGL $AMZN https://t.co/Jpd8MemGxG,722926370280484864,BULLISHBRS,0.0000
12449,1288990142192144385,2020-07-31 00:09:32,$AAPL becoming more of an ESG play than most r...,455309376,Stephanie_Link,0.3400
12450,1288989913954897921,2020-07-31 00:08:38,It’s the Woodstock of tech earnings! $AMZN $FB...,39853973,timseymour,0.6696
12451,1288989006059974658,2020-07-31 00:05:01,Big tech gets bigger in after-hours trading. $...,817007725666242561,MarketRebels,0.0000


#### Same FOR Tesla:

In [10]:
tsla_tweets['rating'] = [analyser.polarity_scores(x)['compound'] for x in tsla_tweets['tweet']]
tsla_tweets['neu'] = [analyser.polarity_scores(x)['neu'] for x in tsla_tweets['tweet']]
tsla_tweets['pos'] = [analyser.polarity_scores(x)['pos'] for x in tsla_tweets['tweet']]
tsla_tweets['neg'] = [analyser.polarity_scores(x)['neg'] for x in tsla_tweets['tweet']]

tsla_tweets[['id','date','tweet','user_id','username','rating']]

,id,date,tweet,user_id,username,rating
0,1324058897510924288,2020-11-04 18:40:15,Biden is likely to win and ev is down! What th...,1271230530571272192,HadiHNCA,0.6580
1,1324058868347973634,2020-11-04 18:40:08,Does anyone also feel $TSLA should be up with ...,26333823,BrianGriffo,0.0000
2,1324054651474374659,2020-11-04 18:23:23,My thoughts on the stock market and the #Elect...,349249475,GerberKawasaki,0.0000
3,1324051821585858565,2020-11-04 18:12:08,"""Einstein was great, but did not succeed at en...",2531643876,MelaynaLokosky,-0.2244
4,1324046888580964352,2020-11-04 17:52:32,Tesla Giga Berlin 🇩🇪 1st Birdseye View As The ...,1689516060,vincent13031925,0.0000
...,...,...,...,...,...,...
18573,1289228332064628738,2020-07-31 15:56:01,$TSLA can securitize all vehicle &amp; solar l...,732548050728701953,ValueAnalyst1,0.0772
18574,1289225297938673664,2020-07-31 15:43:58,I have little doubt Tesla and $TSLA will be le...,546715410,HeartDocTesla,0.6697
18575,1289224836405846016,2020-07-31 15:42:08,$Tsla CEO Elon Musk said in an interview with ...,2704294333,DeItaone,0.3182
18576,1289224311774027778,2020-07-31 15:40:03,"If $TSLA gets added to the S&amp;P 500, it's g...",14096763,TheStalwart,0.0531


#### SAME FOR S&P500:

In [46]:
spx_tweets['rating'] = [analyser.polarity_scores(x)['compound'] for x in spx_tweets['tweet']]
spx_tweets['neu'] = [analyser.polarity_scores(x)['neu'] for x in spx_tweets['tweet']]
spx_tweets['pos'] = [analyser.polarity_scores(x)['pos'] for x in spx_tweets['tweet']]
spx_tweets['neg'] = [analyser.polarity_scores(x)['neg'] for x in spx_tweets['tweet']]

spx_tweets[['id','date','tweet','user_id','username','rating']]

,id,date,tweet,user_id,username,rating
0,1324137715953242118,2020-11-04 23:53:27,"get ready for $spx sell off, insiders will be dumping shares in big chunks instead of doing it smaller groups like they used to bc of the new tax plan. have to be open minded",1091785912667852801,shaurya123k,0.5994
1,1324137066163306496,2020-11-04 23:50:52,"""Truth is exact correspondence with reality. ""- Paramahansa Yogananda #softwareengineer #javascript #python #C++ #golang #nodejs #reactjs #stockopertor #highgrowthstocks $SPX $QQQ $RUT $VIX #canslim #100DaysOfCode",1243665419501744133,glitchbotio,0.3182
2,1324136480613126149,2020-11-04 23:48:32,"How to play the US election? Why I think Australian equities can outperform from here. $COMP, $SPX, $DAX, $N225, $XJO, $AAC, $WPL, $STO, $ORG, $REG, https://t.co/owGy0fyWdt",3133926638,GloverGary,0.3400
3,1324134745878519808,2020-11-04 23:41:39,"*FREE trade plan* 11/5 1. I am bearish as long as below 3480, targets 3369 2. I WONT consider bullish bets if RTH open &gt; 3480 - this is a sell 3. Now 3437 #ES_F $SPX $NDX $SPY $NQ_F",1283357506383876097,AntiTrader94,0.6166
4,1324134387903025156,2020-11-04 23:40:13,"""We need never be ashamed of our tears.""- Charles Dickens #softwareengineer #javascript #python #C++ #golang #nodejs #reactjs #stockopertor #highgrowthstocks $SPX $QQQ $RUT $VIX #canslim #100DaysOfCode",1243665419501744133,glitchbotio,0.1665
...,...,...,...,...,...,...
18737,1301594238224793600,2020-09-03 18:53:43,MARGIN CALL $SPX,31064165,Fxhedgers,0.0000
18738,1301594055038636034,2020-09-03 18:52:59,"$SPX There's slightly more than an hour to go and even if I wanted to have a clue, I don't. $SPX $NDX $IWM https://t.co/Yy6JUzlKYn",202201906,KASDad,0.0000
18739,1301593825962536960,2020-09-03 18:52:05,$spx https://t.co/H3rONzLX3Y,382686379,dimitrisnatsis,0.0000
18740,1301593647666749440,2020-09-03 18:51:22,Stay humble friends. $SPY $SPX https://t.co/mGYWuLyEWV,927754191044362240,BearishBullsCEO,0.4767


In [11]:
aapl_tweets['date'] = pd.to_datetime(aapl_tweets['date'], errors='coerce')
aapl_tweets['Year'] = pd.DatetimeIndex(aapl_tweets['date']).year
aapl_tweets.loc[aapl_tweets.Year != 2020,]

,Unnamed: 0,id,conversation_id,date,place,tweet,language,hashtags,cashtags,user_id,...,nlikes,nreplies,nretweets,quote_url,lang,rating,neu,pos,neg,Year
1367,$BTDG $AMZN $AAPL https://t.co/1WpmzPnLU7,en,[],NaT,f4ef43,FraTa,3,12,https://twitter.com/f4ef43/status/132142968065...,"['https://www.otcmarkets.com/stock/BTDG/news',...",...,NaN,NaN,NaN,NaN,None,0.0,1.0,0.0,0.0,NaN


#### Same FOR Tesla:

In [12]:
tsla_tweets['date'] = pd.to_datetime(tsla_tweets['date'], errors='coerce')
tsla_tweets['Year'] = pd.DatetimeIndex(tsla_tweets['date']).year
tsla_tweets.loc[tsla_tweets.Year != 2020,]

,Unnamed: 0,id,conversation_id,date,place,tweet,language,hashtags,cashtags,user_id,...,nlikes,nreplies,nretweets,quote_url,lang,rating,neu,pos,neg,Year


In [13]:
aapl_tweets['date'] = pd.to_datetime(aapl_tweets.date, format='%Y-%m-%d')
aapl_tweets['day_of_the_week'] = aapl_tweets['date'].dt.day_name()
aapl_tweets[['date','day_of_the_week']]

,date,day_of_the_week
0,2020-11-04 20:14:04,Wednesday
1,2020-11-04 20:12:23,Wednesday
2,2020-11-04 19:28:35,Wednesday
3,2020-11-04 19:21:31,Wednesday
4,2020-11-04 19:11:27,Wednesday
...,...,...
12448,2020-07-31 00:14:39,Friday
12449,2020-07-31 00:09:32,Friday
12450,2020-07-31 00:08:38,Friday
12451,2020-07-31 00:05:01,Friday


In [14]:
tsla_tweets['date'] = pd.to_datetime(tsla_tweets.date, format='%Y-%m-%d')
tsla_tweets['day_of_the_week'] = tsla_tweets['date'].dt.day_name()
tsla_tweets[['date','day_of_the_week']]

,date,day_of_the_week
0,2020-11-04 18:40:15,Wednesday
1,2020-11-04 18:40:08,Wednesday
2,2020-11-04 18:23:23,Wednesday
3,2020-11-04 18:12:08,Wednesday
4,2020-11-04 17:52:32,Wednesday
...,...,...
18573,2020-07-31 15:56:01,Friday
18574,2020-07-31 15:43:58,Friday
18575,2020-07-31 15:42:08,Friday
18576,2020-07-31 15:40:03,Friday


#### Normalizing data:

In [40]:
aapl_tweets.loc[aapl_tweets['neu'] == 1,"rating"] = 0.5 
#Some tweets got polarity neu =1 means that compound will be 0, to avoid that we assign to this values middle_value
aapl_tweets.head()

,Unnamed: 0,id,conversation_id,date,place,tweet,language,hashtags,cashtags,user_id,user_id_str,username,name,link,urls,thumbnail,retweet,nlikes,nreplies,nretweets,quote_url,lang,rating,neu,pos,neg,Year,day_of_the_week,st_date
0,0,1324082507441147906,1324082507441147906,2020-11-04 20:14:04,NaN,I’ll do a video after the close about how do u stay with a “potential new active sequence”. How does a stock stay special to manage it. $aapl is one example to see if it continues. https://t.co/GlXDpqQrwm,en,[],['aapl'],455824771,455824771,RedDogT3,Scott Redler,https://twitter.com/RedDogT3/status/1324082507441147906,[],https://pbs.twimg.com/media/EmAWBY4WkAA62Hm.jpg,False,27.0,0.0,3.0,NaN,en,0.6597,0.866,0.134,0.000,2020.0,Wednesday,2020-11-04
1,1,1324082082985947142,1324082082985947142,2020-11-04 20:12:23,NaN,"Tech stocks among those surging post election. Here's why @RyanDetrick, Senior Market Strategist with @LPL says to stick with the big cap technology names for future growth and outperformance from your portfolio. $FB $GOOGL $AMZN $AAPL https://t.co/WWAvnOmhNe",en,[],"['fb', 'googl', 'amzn', 'aapl']",25360971,25360971,PowerLunch,Power Lunch,https://twitter.com/PowerLunch/status/1324082082985947142,[],https://pbs.twimg.com/amplify_video_thumb/1324081886122135554/img/3LYOQ1nFbMuvyozU.jpg,False,15.0,2.0,6.0,NaN,en,0.3818,0.933,0.067,0.000,2020.0,Wednesday,2020-11-04
2,2,1324071060220727296,1324071060220727296,2020-11-04 19:28:35,NaN,"Have you guys met our options trader ANDY? He has alerted over 1,500%+ in gains to our members TODAY... WOW 🤯🔥 $NIO - 60% ✅ $GWPH - 36% ✅ $MSFT - 225% ✅ $FB - 360% ✅ $SAVA - 60% ✅ $SQ - 251% ✅ $MSFT - 400% ✅ $SPX - 112% ✅ $AAPL - 50% ✅ $ZEN - 191% ✅ $FB - 520% ✅ https://t.co/KpubSYm1Pj",en,[],"['nio', 'gwph', 'msft', 'fb', 'sava', 'sq', 'msft', 'spx', 'aapl', 'zen', 'fb']",1271899909675859968,1271899909675859968,MrFourToEight,FourToEight™,https://twitter.com/MrFourToEight/status/1324071060220727296,[],https://pbs.twimg.com/media/EmALm26U4AEhj7N.png,False,26.0,2.0,2.0,NaN,en,0.6739,0.913,0.065,0.022,2020.0,Wednesday,2020-11-04
3,3,1324069281735176193,1324069281735176193,2020-11-04 19:21:31,NaN,"Remember when people freaked a bit when $AAPL $AMZN $GOOGL $FB reported and three of them were down after great reports? Remember perspective? A day is nothing. That means that day, and today. Things will get bad. But they will get better after they get bad.",en,[],"['aapl', 'amzn', 'googl', 'fb']",100024370,100024370,OphirGottlieb,Ophir Gottlieb,https://twitter.com/OphirGottlieb/status/1324069281735176193,[],NaN,False,102.0,6.0,7.0,NaN,en,-0.3736,0.727,0.114,0.159,2020.0,Wednesday,2020-11-04
4,5,1324066749642399744,1324066749642399744,2020-11-04 19:11:27,NaN,$AAPL https://t.co/E70iPUMuqJ,pl,[],['aapl'],1084979915072700422,1084979915072700422,BBGMarket,Al,https://twitter.com/BBGMarket/status/1324066749642399744,[],https://pbs.twimg.com/media/EmAHqsYXYAEXIB6.jpg,False,6.0,1.0,0.0,NaN,en,0.5000,1.000,0.000,0.000,2020.0,Wednesday,2020-11-04


In [28]:
tsla_tweets.loc[tsla_tweets['neu'] == 1,"rating"] = 0.5 
#Some tweets got polarity neu =1 means that compound will be 0, to avoid that we assign to this values middle_value
pd.set_option('display.max_colwidth', 10000)
pd.set_option('display.max_rows',500)
tsla_tweets[['tweet','rating']].loc[(tsla_tweets['rating'] < 0) &(tsla_tweets['rating'] >= -1)] ##Retrieving negative tweets

,tweet,rating
3,"""Einstein was great, but did not succeed at engineering (and he tried),"" Tesla CEO Elon Musk, who is NOT an engineer. $TSLA https://t.co/S9t4OlAOm3",-0.2244
5,Demand For New Battery Tech Broader Than Election | WSJ $TSLA https://t.co/89ATWuVPnA,-0.1280
6,"For those complaining about $TSLA ‘s performance today: Don’t you think you’re being a bit harsh? TSLA up 6x over the past year, 4x YTD, and almost a triple over the past 6 months. Buy and hold. https://t.co/O7N1JwqcYE",-0.5719
9,$TSLA catalysts - Rev 11/4 1/ Biden will push EV adoption as key plank of Presidency (thru Y.E.) 2/ Moodys credit upgrade (any day) 3/ S&amp;P 500 inclusion (before Y.E.) 4/ CyTruck update (11/30) 5/ FSD full release (12/15) 6/ 4Q Delivs 181K v 168K est (1/4) $720 PT (6-12 mos),-0.4995
16,I strongly disagree. Accelerating EV adoption is an issue all politicians will get behind: 1/ we compete with other global auto mfrs all going EV; 2/ Biden only has to peel off 2 Rep Senators on an issue that carries huge political risk in 2 years if they go against. $tsla,-0.0772
...,...,...
18560,Jsut something to share. I never liked $TSLA I don’t care about company but I’ve become infatuated with trading it’s levels with options. And out of 20 trades I have a max of 3 losers. Losers that I cut early that ended up turning into winners. It’s growing on me.,-0.6504
18564,New documents have been posted in a lawsuit involving $TSLA and our founder. More are likely to be filed later today. The Table of Contents below is from Document 55. https://t.co/rirDmceIqg https://t.co/boGTFMKGmC,-0.2263
18565,This is getting ridiculous. $TSLA,-0.3612
18569,And right back to lows of the day for $TSLA. Pathetic.,-0.6705


##### Converting full date('%Y-%m-%d-%H-%M-%S') to standard format '%Y-%m-%d' <br>Grouping by standart date and calculating daily rating mean:

In [17]:
aapl_tweets['st_date'] = pd.to_datetime(aapl_tweets.date, format='%Y-%m-%d').dt.date #Standart date check
grouped = aapl_tweets[['st_date','rating']]
grouped = grouped.groupby(by = ['st_date'],as_index = False).mean()
#grouped['rating']
grouped.rating

0     0.367810
1     0.297523
2     0.440034
3     0.367997
4     0.279481
        ...   
92    0.352939
93    0.273585
94    0.315867
95    0.373515
96    0.361415
Name: rating, Length: 97, dtype: float64

In [18]:
tsla_tweets['st_date'] = pd.to_datetime(tsla_tweets.date, format='%Y-%m-%d').dt.date #Standart date check
grouped_tsla = tsla_tweets[['st_date','rating']]
grouped_tsla = grouped_tsla.groupby(by = ['st_date'],as_index = False).mean()
#grouped['rating']
grouped_tsla

,st_date,rating
0,2020-07-31,0.238801
1,2020-08-01,0.296005
2,2020-08-02,0.339435
3,2020-08-03,0.346852
4,2020-08-04,0.281988
...,...,...
92,2020-10-31,0.321818
93,2020-11-01,0.244270
94,2020-11-02,0.277849
95,2020-11-03,0.289728


##### Specifying date range for Apple,Tesla Stock market historical data:

In [19]:
df1 = df1.loc[(df1['Date'] >= '2020-07-31')  & (df1['Date'] <= '2020-11-05')]
df3 = df3.loc[(df1['Date'] >= '2020-07-31')  & (df3['Date'] <= '2020-11-05')]

In [20]:
fig = make_subplots(rows = 3,cols = 1,specs = [[{"secondary_y":True}],
                                               [{"secondary_y":True}],
                                               [{"secondary_y":True}],
                                               ])
fig.add_trace(go.Scatter(x = grouped['st_date'],y = grouped['rating'],name = 'AAPL_sent',mode= 'markers+lines', marker = {'color': grouped['rating'], 'colorscale':'RdBu','size':8}))
fig.add_trace(go.Scatter(x = df1['Date'],y = df1["Open"],name = '$AAPL'),secondary_y= True)


fig.add_trace(go.Scatter(x = grouped_tsla['st_date'],y = grouped_tsla['rating'],name = 'TSLA_sent',mode= 'markers+lines', marker = {'color': grouped_tsla['rating'], 'colorscale':'RdBu','size':8}),row=2,col=1)
fig.add_trace(go.Scatter(x = df3['Date'],y = df3["Open"],name = '$TSLA'),secondary_y= True,row=2,col=1)

fig.update_layout(height = 800,paper_bgcolor = 'rgba(0,0,0,0)',plot_bgcolor = 'rgba(0,0,0,0)')
fig.show()

### Counting tweets related for both Tesla and Apple and grouping them by weekday:

In [53]:
from pandas.api.types import CategoricalDtype

aapl_weekday_stat = aapl_tweets[['day_of_the_week','tweet']]
tesla_weekday_stat = tsla_tweets[['day_of_the_week','tweet']]
merged_weekday_stat = pd.concat([aapl_weekday_stat,tesla_weekday_stat],ignore_index=True)
weekdays = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
merged_weekday_stat = merged_weekday_stat.groupby(by = ['day_of_the_week']).count().reindex(weekdays)
merged_weekday_stat['tweet']

day_of_the_week
Monday       5493
Tuesday      5921
Wednesday    4985
Thursday     5071
Friday       5484
Saturday     2075
Sunday       2001
Name: tweet, dtype: int64

#### Graph showing us number of tweets for each day of the week related to $TSLA

In [56]:
fig = px.bar(merged_weekday_stat,x = merged_weekday_stat['tweet'].index, y = ['tweet']
                        ,color = 'value',color_continuous_scale= px.colors.sequential.Bluered,range_color = [merged_weekday_stat['tweet'].min(),merged_weekday_stat['tweet'].max()])

fig.show()

#### Interactive** Polarity distribution radar chart for every ticker

In [51]:
fig = make_subplots(specs=[[{'type':'polar'}]])

categories = ['negative','neutral','positive']

fig.add_trace(go.Scatterpolar(
name = 'TSLA',
r = [
    len(tsla_tweets[['tweet','rating']].loc[(tsla_tweets['rating'] < 0) &(tsla_tweets['rating'] >= -1)]),
    len(tsla_tweets[['tweet','rating']].loc[(tsla_tweets['rating'] >= 0) &(tsla_tweets['rating'] < 0.5)]),
    len(tsla_tweets[['tweet','rating']].loc[(tsla_tweets['rating'] > 0.5) &(tsla_tweets['rating'] <= 1)])
],
    theta = categories,
),1,1)
fig.add_trace(go.Scatterpolar(
name = 'APPL',
r = [
    len(aapl_tweets[['tweet','rating']].loc[(aapl_tweets['rating'] < 0) &(aapl_tweets['rating'] >= -1)]),
    len(aapl_tweets[['tweet','rating']].loc[(aapl_tweets['rating'] >= 0) &(aapl_tweets['rating'] < 0.5)]),
    len(aapl_tweets[['tweet','rating']].loc[(aapl_tweets['rating'] > 0.5) &(aapl_tweets['rating'] <= 1)])
],
    theta = categories,
),1,1)
fig.add_trace(go.Scatterpolar(
name = 'S&P500',
r = [
    len(spx_tweets[['tweet','rating']].loc[(spx_tweets['rating'] < 0) &(tsla_tweets['rating'] >= -1)]),
    len(spx_tweets[['tweet','rating']].loc[(spx_tweets['rating'] > 0) &(tsla_tweets['rating'] < 0.5)]),
    len(spx_tweets[['tweet','rating']].loc[(spx_tweets['rating'] > 0.5) &(tsla_tweets['rating'] <= 1)])
],
    theta = categories,
),1,1)
fig.update_traces(fill='toself')
fig.show()